## https://colab.research.google.com/drive/1BL7nbsp4Ud78WdnQ7X4IGghnMyEArZXJ?usp=drive_link

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

import numpy as np
import matplotlib.pyplot as plt

## 히스토그램 평활화

In [ ]:
img = cv2.imread('/content/drive/MyDrive/패턴인식/PR201910769/images/lab02_zebra.jpg', cv2.IMREAD_GRAYSCALE)
print(img.shape)

In [ ]:
cv2_imshow(img)

In [ ]:
plt.hist(img.flatten(), bins=256);

In [ ]:
imgEqualize = cv2.equalizeHist(img)
plt.hist(imgEqualize.flatten(), bins=256);
plt.show();
cv2_imshow(imgEqualize)

## 히스토그램 역투영

In [ ]:
targetimg = cv2.imread("/content/drive/MyDrive/패턴인식/PR201910769/images/lab02_person.jpg")
modelimg = cv2.imread("/content/drive/MyDrive/패턴인식/PR201910769/images/lab02_modelpatch.jpg")

In [ ]:
cv2_imshow(targetimg)

In [ ]:
cv2_imshow(modelimg)

In [ ]:
## BGR 채널에서 G, R 사용
modelhist = cv2.calcHist([modelimg], [1,2], None, [256, 256], [0,256,0,256])
plt.pcolor(modelhist)
plt.colorbar()
plt.show()

In [ ]:
## HSV 채널에서 H, S 사용
'''
이미지 modelimg를 HSV 채널로 변경하고
modelhist_hsv 에 H와 S 채널 히스토그램 만들기

'''
modelimg_hsv = cv2.cvtColor(modelimg, cv2.COLOR_BGR2HSV)
modelhist_hsv = cv2.calcHist([modelimg_hsv], [0,1], None, [180, 256], [0, 180, 0, 256])

plt.pcolor(modelhist_hsv)
plt.colorbar()
plt.show()

In [ ]:
## YCbCr 채널에서 Cb, Cr 사용
'''
이미지 modelimg를 YCbCr 채널로 변경하고
modelhist_ycbcr 에 Cb와 Cr 채널 히스토그램 만들기

'''
modelimg_ycbcr = cv2.cvtColor(modelimg, cv2.COLOR_BGR2YCrCb)
modelhist_ycbcr = cv2.calcHist([modelimg_ycbcr], [1, 2], None, [256, 256], [0, 256, 0, 256])

plt.pcolor(modelhist_ycbcr)
plt.colorbar()
plt.show()

In [ ]:
targetimg_hsv = cv2.cvtColor(targetimg, cv2.COLOR_BGR2HSV)
targethist_hsv = cv2.calcHist([targetimg_hsv], [0,1], None, [180, 256], [0, 180, 0, 256])

plt.pcolor(targethist_hsv)
plt.colorbar()
plt.show()

In [ ]:
## Model과 target 이미지의 histogram 정규화

modelhist_hsv = modelhist_hsv/modelimg.size
targethist_hsv = targethist_hsv/targetimg.size

## 최대 값이 1보다 작음
print("Max value of Model hist {}".format(modelhist_hsv.max()))
print("Max value of Target hist {}".format(targethist_hsv.max()))

In [ ]:
## 비율 히스토그램 생성
hist_r = modelhist_hsv/ (targethist_hsv + 1e-7)
hist_r = np.minimum(hist_r, 1.0)

In [ ]:
hist_r.shape

In [ ]:
## 출력 이미지 저장을 위한 배열 생성
height, width = targetimg.shape[0], targetimg.shape[1]
output = np.zeros_like(targetimg, dtype='float32')

In [ ]:
for i in range(height):
  for j in range(width):
    '''
    비율 히스토그램을 통해
    각 pixel에 대한 confidence 구하기

    '''
    h_val = targetimg_hsv[i, j, 0]
    s_val = targetimg_hsv[i, j , 1]
    confidence = hist_r[h_val, s_val]
    output[i,j] = confidence

In [ ]:
ret, masked = cv2.threshold(output, 0.02, 255, cv2.THRESH_BINARY)
cv2_imshow(masked)